<div style="background-color: white; color: #252127; text-align: center;">   
    <br>
    <h3> UNIVERSIDAD NACIONAL AUTÓNOMA DE MÉXICO </h3>
    <h3> FACULTAD DE ESTUDIOS SUPERIORES ACATLÁN </h3>
    <h2 style="color: #2d572c!important"> "Comparación entre procesamiento paralelo y secuencial" </h2>
     <br>
 
</div>

In [96]:
import numpy as np
from sympy import Matrix, gcd
from concurrent.futures import ThreadPoolExecutor
import random
from unidecode import unidecode 

In [122]:
# Función para multiplicación de matrices paralela
def parallel_matrix_mult(a, b):
    if a.shape[1] != b.shape[0]:
        raise ValueError("Las matrices no pueden ser multiplicadas")

    result = np.zeros((a.shape[0], b.shape[1]))

    def compute_element(x, y):
        result[x][y] = np.dot(a[x, :], b[:, y])

    with ThreadPoolExecutor(max_workers=2) as executor:
        for i in range(a.shape[0]):
            for j in range(b.shape[1]):
                executor.submit(compute_element, i, j)

    return result

In [123]:
def multiplicar_matrices(a, b):
    if a.shape[1] != b.shape[0]:
        raise ValueError("Las matrices no pueden ser multiplicadas")

    result = np.zeros((a.shape[0], b.shape[1]))

    for i in range(a.shape[0]):
        for j in range(b.shape[1]):
            result[i][j] = np.dot(a[i, :], b[:, j])

    return result


# Funciones

In [124]:
# Función para convertir el texto en una matriz numérica
def text_to_matrix(text,alphabet):
    text = text.lower()
    matrix = []
    for char in text:
        if char in alphabet:
            matrix.append(alphabet.index(char))
    return np.array(matrix)

In [125]:
# Función para convertir la matriz numérica de vuelta a texto
def matrix_to_text(matrix,alphabet):
    text = ""
    for number in matrix:
        text += alphabet[int(number)]
    return text

In [126]:
# Función para cifrar el texto con una matriz clave
def hill_cipher(text, key_matrix,alphabet,metodo):
    text_matrix = text_to_matrix(text,alphabet)
    n = key_matrix.shape[0]
    # Aseguramos que el tamaño del texto es divisible por el tamaño de la matriz clave
    while len(text_matrix) % n != 0:
        text_matrix = np.append(text_matrix, 0)
    text_matrix = text_matrix.reshape(-1, n)
    if(metodo == "Paralelo"):
        encrypted_matrix = parallel_matrix_mult(key_matrix, text_matrix.T).T % len(alphabet)
    else:
        encrypted_matrix = multiplicar_matrices(key_matrix, text_matrix.T).T % len(alphabet)
    return matrix_to_text(encrypted_matrix.flatten(),alphabet)

In [127]:
# Función para descifrar el texto con una matriz clave
def hill_decipher(text, key_matrix,alphabet, metodo):
    text_matrix = text_to_matrix(text,alphabet)
    n = key_matrix.shape[0]
    text_matrix = text_matrix.reshape(-1, n)
    # Calculamos la inversa de la matriz en modulo 26
    key_matrix_inv = Matrix(key_matrix).inv_mod(len(alphabet))
    if(metodo == "Paralelo"):
  
        decrypted_matrix = parallel_matrix_mult(key_matrix_inv, text_matrix.T).T % len(alphabet)
    else:
  
        decrypted_matrix = multiplicar_matrices(key_matrix_inv, text_matrix.T).T % len(alphabet)
    return matrix_to_text(decrypted_matrix.flatten(),alphabet)

In [128]:
def generate_matrix(text):
    n = len(text)
    matrix = np.zeros((n, n), dtype=int)

    for i in range(n):
        matrix[i][i] = 3  

    return matrix

In [129]:
def funcion_proyecto(text, key_matrix, metodo):
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    encrypted_text = hill_cipher(text, key_matrix,alphabet,metodo)
    decrypted_text = hill_decipher(encrypted_text, key_matrix,alphabet,metodo)
    return encrypted_text,decrypted_text

In [136]:
import time

def main():
    frase = "Hole hola hola hola hola hola Hola hola hola hola"
    frase = frase.replace(" ", "")
    key_matrix = generate_matrix(frase)

    # Calcula el tiempo de la ejecución paralela
    start_time = time.time()
    resultado_paralelo = funcion_proyecto(frase, key_matrix, "Paralelo")
    paralelo_time = time.time() - start_time
    print(f"El tiempo de ejecución en paralelo fue: {paralelo_time} segundos")

    # Calcula el tiempo de la ejecución secuencial
    start_time = time.time()
    resultado_secuencial = funcion_proyecto(frase, key_matrix, "Secuencial")
    secuencial_time = time.time() - start_time
    print(f"El tiempo de ejecución secuencial fue: {secuencial_time} segundos")


In [137]:
main()

El tiempo de ejecución en paralelo fue: 11.95670747756958 segundos
El tiempo de ejecución secuencial fue: 12.127100944519043 segundos
